In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Preproccesing and recostructing a dataset from DLS
Few lines intro

**Learning objectives:**
1. Centre of rotation
2. Reizer
3. reconstructing a real dataset

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from ccpi.framework import ImageData, ImageGeometry
from ccpi.framework import AcquisitionGeometry, AcquisitionData

from ccpi.optimisation.algorithms import CGLS, SIRT
from ccpi.optimisation.functions import Norm2Sq, L1Norm
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity
from ccpi.framework import BlockDataContainer

from ccpi.processors import Resizer, CenterOfRotationFinder

from ccpi.io import NEXUSDataReader
from ccpi.astra.operators import AstraProjectorSimple , AstraProjector3DSimple

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy
from utilities import islicer
from utilities import plotter2D


### Read in the dataset

About the data?

In [ ]:
## Set up a reader object pointing to the Nexus data set. Revise path as needed.
# The data is already  corrected for by flat and dark field.
path = os.path.join(sys.prefix, 'share','ccpi','24737_fd_normalised.nxs')
myreader = NEXUSDataReader(nexus_file=path)
data_raw = myreader.load_data()

In [ ]:
#Look at the data set
print(type(data_raw))
print(data_raw)
islicer(data_raw, direction=0)

### Use Resizer() to pre-proccess the data

We want to remove the top row data with the dark pixel

Use the processor Resizer()


In [ ]:
start_row = 1
end_row = data_raw.shape[1]
roi_crop = [-1,(start_row, end_row),-1]

In [ ]:
resizer = Resizer(roi=roi_crop)
resizer.set_input(data_raw)
data_cropped = resizer.get_output()

In [ ]:
#Look at the data set
print(type(data_cropped))
print(data_cropped)
islicer(data_cropped, direction=0)

### Use CenterOfRotationFinder()

In [ ]:
# Set up CenterOfRotationFinder object to center data.
# Set the output of the normaliser as the input and execute to determine center.
cor = CenterOfRotationFinder()
cor.set_input(data_cropped)
center_of_rotation = cor.get_output()
print( "centre of rotation at pixel: ", center_of_rotation)

In [ ]:
#exercise use the resizer to crop the image

In [ ]:
#get image width
num_pixels_y = data_cropped.shape[2]
cor_shift = center_of_rotation - num_pixels_y/2

In [ ]:
start_column = int(round(2.*cor_shift))
end_column = data_cropped.shape[2]

roi_crop = [-1,-1,(start_column, end_column)]
resizer = Resizer(roi=roi_crop)
resizer.set_input(data_cropped)
data_centred = resizer.get_output()

In [ ]:
#Look at the data set
print(type(data_centred))
print(data_centred)
islicer(data_centred, direction=0)

### Set up the data ready for ASTRA

ASTRA expects the data in the order `['vertical','angle','horizontal']` so we need to permute the dataset

In [ ]:
print(data_centred)

In [ ]:
data = data_centred.subset(dimensions=['vertical','angle','horizontal'])

In [ ]:
print(data)

ASTRA takes the angles as radians so let's convert the angles geometry data

In [ ]:
#convert the angles to radians
if(data.geometry.angle_unit == 'degree'):
    data.geometry.angle_unit = 'radian'
    data.geometry.angles = data.geometry.angles * np.pi /180.

And finally convert from intensity to attenuation data

In [ ]:
#Convert from intensity to attenuation data
#how can I make sure thy don't execute this more than once?
data.fill(-np.log(data.as_array()))
#Look at the data set
print(type(data))
print(data)
islicer(data, direction=1) #remember we've transposed the data so angles is now the 2nd axis

### Define the Geometry

#### Acquistion geometry
In the 2D example we used:<br>
`ag = AcquisitionGeometry(geom_type='parallel', dimension='2D', angles=angles, pixel_num_h=number_pixels_x)`<br>

For 3D we need to change the dimension description to ` dimension='3D'`, and pass the number of vertical pixels as `pixel_num_v`<br>

However we've been using the acquistion geometry throughout this notebook so let's just clone the version we've already set up.

#### Image geometry
In the 2D example we used:<br>
`ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)`

For ad 3D reconstruction we also need to pass the number of voxels we want in the $z$-direction as `voxel_num_z`

If you create the image geometry from the acquisiton geometry this is set to `pixel_num_h` by default.

In [ ]:
# Create Acquisition Geometry
ag = data.geometry.clone()

# Create Image Geometry
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                   voxel_num_y=ag.pixel_num_h, 
                   voxel_num_z=ag.pixel_num_v)

### Define the projector

In the 2D example we used the 2D projector from ASTRA<br>
`'AstraProjectorSimple(volume_geometry, sinogram_geometry, device)`

Use ASTRA's 3D projector, note this projector is GPU only<br>
`AstraProjector3DSimple(volume_geometry, sinogram_geometry)`

In [ ]:
# Define the projector object
print ("Define projector")
Cop = AstraProjector3DSimple(ig, ag)

### Run SIRT

In [ ]:
#setup SIRT
x_init = ig.allocate(0)
sirt = SIRT(x_init=x_init, operator=Cop, data=data, update_objective_interval = 10)
sirt.max_iteration = 1000

In [ ]:
#run the algorithm
sirt.run(100, verbose = True)

In [ ]:
#plot the results
SIRT_output = sirt.get_output()
islicer(SIRT_output, direction=0,fix_range=False)

### Run Tikhonov CGLS

In [ ]:
#define the operator
alpha = 5
L = Gradient(ig)
operator_block = BlockOperator( Cop, alpha * L, shape=(2,1))

In [ ]:
#define the data b
data_block = BlockDataContainer(data, L.range_geometry().allocate(0))

In [ ]:
#setup Tikonov
L = Gradient(ig)
alpha = 75

x_init = ig.allocate(0)
cgls_tikhonov = CGLS(x_init=x_init, operator=operator_block, data=data_block, update_objective_interval = 10)
cgls_tikhonov.max_iteration = 1000

In [ ]:
#run the algorithm
cgls_tikhonov.run(100, verbose = True)

In [ ]:
#plot the results
CGLS_tikhonov_output = cgls_tikhonov.get_output()

islicer(CGLS_tikhonov_output, direction=0,fix_range=False)

### Run FBP

In [ ]:
#setup FBP


In [ ]:
#plot the results
#islicer(FBP_output, direction=0,fix_range=False)

### Summary

In [ ]:
#compare the outputs of unregularise and regularised CGLS

islicer(SIRT_output, direction=0,fix_range=False)
islicer(CGLS_tikhonov_output, direction=0,fix_range=False)
#islicer(FBP_output, direction=0,fix_range=False)